In [1]:
import pandas as pd
from gurobipy import *
import numpy as np

In [2]:
data = pd.read_csv('submission_task12.csv')

In [3]:
data['y'] = np.abs(data['y'])
data

,y,existence expectancy index
0,0.042148,0.456086
1,0.039196,0.529835
2,0.039196,0.560976
3,0.038947,0.565910
4,0.030233,0.588274
5,0.039959,0.589654
6,0.020612,0.589852
7,0.042148,0.594438
8,0.041415,0.598516
9,0.033197,0.598777


In [6]:
data = data['existence expectancy index']


In [7]:
data.info()

AttributeError: 'Series' object has no attribute 'info'

1) Index predictions are evaluated using RMSE metric

2) Energy allocation is also evaluated using RMSE metric and has a set of known factors that
need to be taken into account.

Every galaxy has a certain limited potential for improvement in the index described by the
following function:

Potential for increase in the Index = -np.log(Index+0.01)+3

Likely index increase dependent on potential for improvement and on extra energy
availability is described by the following function:

Likely increase in the Index = extra energy * Potential for increase in the Index **2 /
1000

There are also several constraints:

in total there are 50000 zillion DSML available for allocation

no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML

galaxies with low existence expectancy index below 0.7 should be allocated at least
10% of the total energy available


In [4]:
def potential(i,data):
    index = data['y'][i]
    return (((-np.log(index+0.01)+3)**2) / 1000 )

In [5]:
data.reset_index(inplace=True)

In [6]:
data.index

RangeIndex(start=0, stop=890, step=1)

In [7]:
avail = 50000
max = 100.0
min = 0.0

In [8]:
model = Model("Energy Optimizer")
x = model.addVars(data.index, vtype=GRB.CONTINUOUS)
model.setObjective(quicksum(x[i] * potential(i,data)  for i in data.index),GRB.MAXIMIZE)
# constraints 
model.addConstr(quicksum( x[i] for i in data.index) <= avail )
model.addConstrs( x[i] <= max for i in data.index)  
model.addConstrs( x[i] >= min for i in data.index) 
model.addConstr(quicksum( x[i] for i in data[data["existence expectancy index"] < 0.7].index) >= 0.1 * avail )


Using license file C:\Users\User\gurobi.lic
Academic license - for non-commercial use only


<gurobi.Constr *Awaiting Model Update*>

In [9]:
model.write("debugmodel.lp")
with open("debugmodel.lp") as f:
    print(f.read())

\ Model Energy Optimizer
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  0.035446213562923 C0 + 0.0361435675890833 C1 + 0.0361435675890833 C2
   + 0.0362044 C3 + 0.038602140825404 C4 + 0.0359585656863016 C5
   + 0.0420730286355796 C6 + 0.035446213562923 C7 + 0.0356150245942544 C8
   + 0.0377239700593774 C9 + 0.0361836912159997 C10
   + 0.0425708043280396 C11 + 0.035446213562923 C12
   + 0.0355609661659176 C13 + 0.0390980966634678 C14
   + 0.0385836217486196 C15 + 0.0391542556983589 C16
   + 0.0385836217486196 C17 + 0.0361435675890833 C18
   + 0.0361836912159997 C19 + 0.0387632784297665 C20
   + 0.0426600525906895 C21 + 0.036478809460185 C22
   + 0.0359585656863016 C23 + 0.0354469932546755 C24
   + 0.0405649266702558 C25 + 0.0420730286355796 C26
   + 0.0417241530152805 C27 + 0.0355609661659176 C28
   + 0.0377239700593774 C29 + 0.0356709469146878 C30
   + 0.0359585656863016 C31 + 0.0355609661659176 C32
   + 0.0323025878034089 C33 + 0.042361804556

In [10]:
model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 1782 rows, 890 columns and 2736 nonzeros
Model fingerprint: 0xf4e549c3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 4e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+04]
Presolve removed 1780 rows and 656 columns
Presolve time: 0.01s
Presolved: 2 rows, 234 columns, 270 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1924651e+03   6.812500e+03   0.000000e+00      0s
       1    1.8530492e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  1.853049196e+03


In [11]:
l = []
for i in model.getVars():
    l.append(i.x)


In [12]:
l

[100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 0.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 0.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 0.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0

In [13]:
data['pred']= l

In [47]:
l = []
for i in model.getVars():
    l.append(i.x)
data['pred']= l
l = []
for i in data.index:
    l.append(potential(i,data))
data['pot']= l

data['likely'] = data['pred'] * data['pot']
    

In [48]:
vars = model.getVars()

In [49]:
x = vars[0]

In [50]:
print(x)

<gurobi.Var C0 (value 0.0)>


In [51]:
data.pred.min()

0.0

In [52]:
data.pred.max()

100.0

In [17]:
data.pred.describe()

count    890.000000
mean      56.179775
std       49.644532
min        0.000000
25%        0.000000
50%      100.000000
75%      100.000000
max      100.000000
Name: pred, dtype: float64

In [19]:
sum(data.pred)

50000.0

In [19]:
sum(data['likely'])

KeyError: 'likely'

In [24]:
data[['y',"existence expectancy index",'pred','pot','likely']][data["existence expectancy index"] < 0.7]

,y,existence expectancy index,pred,pot,likely
0,0.042004,0.456086,100.0,0.035479,3.547902
1,0.038978,0.529835,100.0,0.036197,3.619682
2,0.034734,0.560976,100.0,0.037296,3.729562
3,0.040033,0.565910,100.0,0.035941,3.594089
4,0.034828,0.588274,100.0,0.037270,3.727001
5,0.043587,0.589654,100.0,0.035123,3.512275
6,0.036471,0.589852,100.0,0.036832,3.683176
7,0.041750,0.594438,100.0,0.035537,3.553750
8,0.038720,0.598516,100.0,0.036260,3.626047
9,0.042313,0.598777,100.0,0.035409,3.540859


In [25]:
data[['y',"existence expectancy index",'pred','pot','likely']][data["existence expectancy index"] < 0.7].sum()

y                                2.438058
existence expectancy index      42.300663
pred                          6400.000000
pot                              2.433058
likely                         236.862696
dtype: float64

In [20]:
# Percentage of energy given to galaxies with existence index < 0.7
data[['pred']][data["existence expectancy index"] < 0.7].sum()/avail *100

pred    37.2
dtype: float64

In [21]:
# Percentage of energt given to 
len(data[(data["existence expectancy index"] < 0.7) & (data.pred > 0)]) / len(data[data["existence expectancy index"] < 0.7]) * 100

27.474150664697195

In [14]:
final_df = data[['y','pred']]

In [15]:
final_df

,y,pred
0,0.042148,100.0
1,0.039196,100.0
2,0.039196,100.0
3,0.038947,100.0
4,0.030233,100.0
5,0.039959,100.0
6,0.020612,100.0
7,0.042148,100.0
8,0.041415,100.0
9,0.033197,100.0


In [16]:
final_df.columns = ['pred','opt_pred']

In [17]:
final_df.index.name = 'index'

In [18]:
final_df

,pred,opt_pred
index,,
0,0.042148,100.0
1,0.039196,100.0
2,0.039196,100.0
3,0.038947,100.0
4,0.030233,100.0
5,0.039959,100.0
6,0.020612,100.0
7,0.042148,100.0
8,0.041415,100.0


In [19]:
final_df.to_csv('fina_submission12.csv')